In [1]:
#conda install seaborn

In [2]:
#conda install -c anaconda line_profiler


In [3]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('..')
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from merf.utils import MERFDataGenerator
from merf.merf import MERF
from merf.viz import plot_merf_training_stats



In [5]:
filepath = 'H:/cloud/cloud_data/Projects/MixedEffectModels/data/forMERF.csv'
cct_data=pd.read_csv(filepath)
#type(cct_data)

In [6]:
cct_data.head()


,patient_ID,dataset,study,intercept,slope,intercept1,slope1,study_no,gender,typical3,atypical2,nongil,age,CT_pos,CT_non,Cath_pos,Cath_non,iv_prot,contrast_amount,contrast_conc,Agatston_score,height,hypertension,diabetes,hyperlipidemia,smoker,pos_fam_hist,prior_myo_inf,typical3_imp,atypical2_imp,nongil_imp,iv_prot_imp,contrast_amount_imp,contrast_conc_imp,height_imp,hypertension_imp,diabetes_imp,hyperlipidemia_imp,smoker_imp,risk,pos_fam_hist_imp,prior_myo_inf_imp
0,50,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,1,0.0,1.0,0.0,54,1,0,1,0,0.0,80,370.0,13.0,170.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0,0,80,370,170.0,0,0,0,1,1,0,0
1,51,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,1.0,0.0,61,0,0,0,0,0.0,70,370.0,21.0,168.0,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,70,370,168.0,0,0,0,1,1,1,0
2,52,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,0.0,1.0,55,0,0,0,0,1.0,70,370.0,0.0,173.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,1,1,70,370,173.0,0,0,0,1,1,0,0
3,53,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,1.0,0.0,67,1,0,1,0,0.0,80,370.0,226.0,152.0,0.0,0.0,0.0,1.0,0.0,1.0,0,1,0,0,80,370,152.0,0,0,0,1,1,0,1
4,54,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,1,0.0,0.0,1.0,57,0,0,0,0,0.0,80,370.0,148.0,169.0,1.0,1.0,1.0,1.0,0.0,0.0,0,0,1,0,80,370,169.0,1,1,1,1,1,0,0


In [7]:

check=pd.isnull(cct_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(check)

      patient_ID  dataset  study  intercept  slope  intercept1  slope1  \
0          False    False  False      False  False       False   False   
1          False    False  False      False  False       False   False   
2          False    False  False      False  False       False   False   
3          False    False  False      False  False       False   False   
4          False    False  False      False  False       False   False   
5          False    False  False      False  False       False   False   
6          False    False  False      False  False       False   False   
7          False    False  False      False  False       False   False   
8          False    False  False      False  False       False   False   
9          False    False  False      False  False       False   False   
10         False    False  False      False  False       False   False   
11         False    False  False      False  False       False   False   
12         False    False  False      

In [8]:

cct_data["Study"]=cct_data["study"].astype('category')
cct_data["study_no"]=cct_data["study_no"].astype('category')


cct_data.head()

,patient_ID,dataset,study,intercept,slope,intercept1,slope1,study_no,gender,typical3,atypical2,nongil,age,CT_pos,CT_non,Cath_pos,Cath_non,iv_prot,contrast_amount,contrast_conc,Agatston_score,height,hypertension,diabetes,hyperlipidemia,smoker,pos_fam_hist,prior_myo_inf,typical3_imp,atypical2_imp,nongil_imp,iv_prot_imp,contrast_amount_imp,contrast_conc_imp,height_imp,hypertension_imp,diabetes_imp,hyperlipidemia_imp,smoker_imp,risk,pos_fam_hist_imp,prior_myo_inf_imp,Study
0,50,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,1,0.0,1.0,0.0,54,1,0,1,0,0.0,80,370.0,13.0,170.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0,0,80,370,170.0,0,0,0,1,1,0,0,Alkadhi_2
1,51,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,1.0,0.0,61,0,0,0,0,0.0,70,370.0,21.0,168.0,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,70,370,168.0,0,0,0,1,1,1,0,Alkadhi_2
2,52,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,0.0,1.0,55,0,0,0,0,1.0,70,370.0,0.0,173.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,1,1,70,370,173.0,0,0,0,1,1,0,0,Alkadhi_2
3,53,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,1.0,0.0,67,1,0,1,0,0.0,80,370.0,226.0,152.0,0.0,0.0,0.0,1.0,0.0,1.0,0,1,0,0,80,370,152.0,0,0,0,1,1,0,1,Alkadhi_2
4,54,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,1,0.0,0.0,1.0,57,0,0,0,0,0.0,80,370.0,148.0,169.0,1.0,1.0,1.0,1.0,0.0,0.0,0,0,1,0,80,370,169.0,1,1,1,1,1,0,0,Alkadhi_2


In [9]:
cct_data.columns

Index(['patient_ID', 'dataset', 'study', 'intercept', 'slope', 'intercept1',
       'slope1', 'study_no', 'gender', 'typical3 ', 'atypical2', 'nongil',
       'age', 'CT_pos', 'CT_non', 'Cath_pos', 'Cath_non', 'iv_prot',
       'contrast_amount', 'contrast_conc', 'Agatston_score', 'height',
       'hypertension', 'diabetes', 'hyperlipidemia', 'smoker', 'pos_fam_hist',
       'prior_myo_inf', 'typical3_imp', 'atypical2_imp', 'nongil_imp',
       'iv_prot_imp', 'contrast_amount_imp', 'contrast_conc_imp', 'height_imp',
       'hypertension_imp', 'diabetes_imp', 'hyperlipidemia_imp', 'smoker_imp',
       'risk', 'pos_fam_hist_imp', 'prior_myo_inf_imp', 'Study'],
      dtype='object')

In [10]:
cct_data['interaction1']=cct_data['Agatston_score']*cct_data['contrast_conc_imp']
cct_data['interaction2']=cct_data['Agatston_score']*cct_data['diabetes_imp']
cct_data['interaction3']=cct_data['Agatston_score']*cct_data['prior_myo_inf_imp']
check=pd.isnull(cct_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(check)


      patient_ID  dataset  study  intercept  slope  intercept1  slope1  \
0          False    False  False      False  False       False   False   
1          False    False  False      False  False       False   False   
2          False    False  False      False  False       False   False   
3          False    False  False      False  False       False   False   
4          False    False  False      False  False       False   False   
5          False    False  False      False  False       False   False   
6          False    False  False      False  False       False   False   
7          False    False  False      False  False       False   False   
8          False    False  False      False  False       False   False   
9          False    False  False      False  False       False   False   
10         False    False  False      False  False       False   False   
11         False    False  False      False  False       False   False   
12         False    False  False      

In [11]:
from sklearn.model_selection import train_test_split
cols = [col for col in cct_data.columns if col not in ['Patient_ID', 'typical3','atypical2', 'nongil'
                                                       , 'CT_non', 'Cath_pos','Cath_non', 'iv_prot', 'contrast_amount' 
                                                       ,'contrast_conc','height', 'hypertension', 'diabetes','hyperlipidemia'
                                                       , 'smoker', 'pos_fam_hist', 'prior_myo_inf']]
train= cct_data[cols]
target= cct_data['Cath_pos']
data_train, data_test, target_train, target_test = train_test_split(train,target, stratify= cct_data['Study'], test_size = 0.20, random_state = 10,)

In [12]:
data_train

,patient_ID,dataset,study,intercept,slope,intercept1,slope1,study_no,gender,typical3,age,CT_pos,Agatston_score,typical3_imp,atypical2_imp,nongil_imp,iv_prot_imp,contrast_amount_imp,contrast_conc_imp,height_imp,hypertension_imp,diabetes_imp,hyperlipidemia_imp,smoker_imp,risk,pos_fam_hist_imp,prior_myo_inf_imp,Study,interaction1,interaction2,interaction3
2318,2368,Gueret_1,Gueret_1,-0.001787,0.092737,-0.049897,0.000623,70,1,1.0,77,1,2686.0,1,0,0,0,80,400,165.00,1,0,1,0,1,1,0,Gueret_1,1074400.0,0.0,0.0
705,755,Garcia_1,Garcia_1,-0.118753,-0.065246,-0.049895,0.001141,13,1,0.0,74,0,0.0,0,1,0,0,86,356,179.00,1,0,0,0,1,0,0,Garcia_1,0.0,0.0,0.0
2275,2325,Gueret_1,Gueret_1,-0.001787,0.092737,-0.049897,0.000623,70,1,0.0,81,0,334.0,0,1,0,0,90,400,175.00,1,0,0,0,1,1,0,Gueret_1,133600.0,0.0,0.0
1295,1345,Mendoza_Rodriguez_1,Mendoza_Rodriguez_1,-0.054726,0.011631,-0.049876,0.005288,34,0,0.0,50,0,0.0,0,1,0,0,80,370,162.00,1,0,1,1,1,1,0,Mendoza_Rodriguez_1,0.0,0.0,0.0
460,510,Diederichsen_1,Diederichsen_1,-0.046911,0.008946,-0.049890,0.002103,11,0,1.0,45,0,0.0,1,0,0,1,100,370,166.10,0,0,1,1,1,1,0,Diederichsen_1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2209,Gueret_1,Gueret_1,-0.001787,0.092737,-0.049897,0.000623,70,1,0.0,61,0,0.0,0,0,1,0,110,350,181.00,1,1,0,0,1,0,0,Gueret_1,0.0,0.0,0.0
46,96,Alkadhi_2,Alkadhi_2,1.599454,1.103524,0.049901,0.000224,2,0,0.0,54,0,0.0,0,0,1,1,80,370,172.00,0,0,0,0,0,1,0,Alkadhi_2,0.0,0.0,0.0
720,770,Garcia_1,Garcia_1,-0.118753,-0.065246,-0.049895,0.001141,13,0,1.0,63,1,35.0,1,0,0,0,88,358,160.00,1,0,1,0,1,0,0,Garcia_1,12530.0,0.0,0.0
482,532,Diederichsen_1,Diederichsen_1,-0.046911,0.008946,-0.049890,0.002103,11,0,1.0,52,0,0.0,1,0,0,1,100,370,167.06,0,0,0,1,1,1,0,Diederichsen_1,0.0,0.0,0.0


In [13]:
#data_train[['intercept','slopeCA']] = cct_data[['intercept','slopeCA']]# this for the calcium score alone#don't use
#data_test[['intercept','slopeCA']] = cct_data[['intercept','slopeCA']]#this for the calcium score alone#don't use
data_train[['intercept','slope']] = cct_data[['intercept','slope']]
data_test[['intercept','slope']] = cct_data[['intercept','slope']]
data_train[['intercept','slopeCA']] = cct_data[['intercept','slopeCA']]# this for the calcium score  and patient data and CT result# alone#don't use
data_test[['intercept','slopeCA']] = cct_data[['intercept','slopeCA']]


H:\ProgramFiles\Anaconda2\envs\env_mem\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


KeyError: "['slopeCA'] not in index"

In [ ]:
data_train.columns

In [ ]:
#training merf on  agatston score plus (patient individual data (PID)
X_train = data_train[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp',
       'diabetes_imp', 'hyperlipidemia_imp', 
       'interaction1', 'prior_myo_inf_imp']]
Z_train = data_train[['intercept','slope']]
clusters_train = data_train['study']
y_train = target_train

mrf = MERF(max_iterations=50)
mrf.fit(X_train, Z_train, clusters_train, y_train)

In [ ]:
plot_merf_training_stats(mrf, num_clusters_to_plot=30)


In [ ]:
X_test = data_test[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp',
       'diabetes_imp', 'hyperlipidemia_imp', 
       'interaction1', 'prior_myo_inf_imp']]
Z_test = data_test[['intercept','slope']]
clusters_test = data_test['study']
y_test = target_test

In [ ]:
y_hat_known = mrf.predict(X_test, Z_test, clusters_test)

#auc rf agatston score plus (patient individual data (PID) by merf
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_hat_known)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for MI classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)
print("AUC:", metrics.roc_auc_score(y_test, y_hat_known))



In [ ]:
#Normal random forest not mixed effect for  agatston score plus (patient individual data (PID) 

X_train_rf = data_train[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp',
       'diabetes_imp', 'hyperlipidemia_imp', 
       'interaction1', 'prior_myo_inf_imp']]
X_test_rf  = data_test[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp',
       'diabetes_imp', 'hyperlipidemia_imp', 
       'interaction1', 'prior_myo_inf_imp']]
y_train_rf = target_train
y_test_rf = target_test

In [ ]:
#not mixed effect
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30)
rf.fit(X_train_rf, y_train_rf)
y_pred_prob_rf = rf.predict_proba(X_test_rf)[:, 1]


In [ ]:
y_hat_known_rf = rf.predict_proba(X_test_rf)[:, 1]
y_pred_class_rf = rf.predict(X_test_rf)


In [ ]:
X_test = data_test[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp',
       'interaction3', 'diabetes_imp', 'hyperlipidemia_imp', 'interaction2',
       'interaction1', 'prior_myo_inf_imp']]
Z_test = data_test[['intercept','slope']]
clusters_test = data_test['study']
y_test = target_test

In [ ]:
#auc rf agat*PID normal rf not merf
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_hat_known_rf)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for MI classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)
print("AUC:", metrics.roc_auc_score(y_test, y_hat_known_rf))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_class_rf))


In [ ]:
import shap

In [ ]:
mrf.trained_fe_model

In [ ]:
#conda install -c conda-forge shap


In [ ]:
#conda install -c conda-forge/label/cf201901 shap


In [ ]:
#conda install -c conda-forge/label/cf202003 shap

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
import shap
shap_values = shap.TreeExplainer(mrf.trained_fe_model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")
#explainer = shap.TreeExplainer(mrf.trained_fe_model)
#shap_values = explainer.shap_values(X_train)


In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_train)

In [ ]:
shap.dependence_plot("Agatston_score", shap_values, X_train)

In [ ]:
# training merf on agatstone score + PID + CT
X_train = data_train[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp','CT_pos',
       'interaction3', 'diabetes_imp', 'hyperlipidemia_imp', 'interaction2',
       'interaction1', 'prior_myo_inf_imp']]
Z_train = data_train[['interceptCT','slopeCT']]
clusters_train = data_train['study']
y_train = target_train

mrf = MERF(max_iterations=50)
mrf.fit(X_train, Z_train, clusters_train, y_train)

In [ ]:
# AUC for agatstone score + PID + CT

X_test = data_test[['gender', 'age','Agatston_score','typical3_imp',
       'contrast_conc_imp','CT_pos',
       'interaction3', 'diabetes_imp', 'hyperlipidemia_imp', 'interaction2',
       'interaction1', 'prior_myo_inf_imp']]
Z_test = data_test[['interceptCT','slopeCT']]
clusters_test = data_test['study']
y_test = target_test

In [ ]:
y_hat_known = mrf.predict(X_test, Z_test, clusters_test)

#auc rf agat*PID + CT by merf
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_hat_known)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for MI classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)
print("AUC:", metrics.roc_auc_score(y_test, y_hat_known))


In [ ]:
X_test = data_test[['CT_pos']]
clusters_test = data_test['study']
Z_test = data_test[['intercept','slope']]
y_test = target_test

In [ ]:
X_train = data_train[['CT_pos']]
Z_train = data_train[['interceptCT','slopeCT']]
clusters_train = data_train['study']
y_train = target_train

mrf = MERF(max_iterations=50)
mrf.fit(X_train, Z_train, clusters_train, y_train)

In [ ]:
X_test = data_test[['CT_pos']]
Z_test = data_test[['interceptCT','slopeCT']]
clusters_test = data_test['study']
y_test = target_test

In [ ]:
y_hat_known = mrf.predict(X_test, Z_test, clusters_test)

#auc rf agat*PID merf
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_hat_known)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for MI classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)
print("AUC:", metrics.roc_auc_score(y_test, y_hat_known))
